In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import os
import ase
import tqdm
import pickle
import numpy as np
import ase.io as aio

from toolz.curried import pipe, curry, compose

import warnings
warnings.filterwarnings('ignore')

In [14]:
import plot
import atommks.helpers as utils
import atommks.atomstat as astat

In [15]:
from ase import Atoms 
get_atoms = lambda a: Atoms(["Cu"]*len(a),
                            positions=a.get_positions(),
                            cell=a.get_cell(), 
                            pbc=[1, 1, 1], 
                            tags=a.get_tags())

runs = np.loadtxt("runs.txt").astype(int)

## Computation over all structures

In [16]:
folder_rdf = "data/stats_new_irr_local"
if not os.path.exists(folder_rdf):
    os.mkdir(folder_rdf)

In [17]:
cutoff = 7.0
r_stat = 5.0
len_pixel = 10.0 

In [18]:
@curry
def generate_stats(run, r_stat, len_pixel):
    
    atoms = pipe("data/structs/dump.arc.{}_atom.pkl".format(run), 
             lambda f: utils.load_file(f), 
             lambda a: get_atoms(a))

    idx = np.where(atoms.get_tags() != 1)[0]
    print("total atoms: {}, disturbed atoms: {}".format(len(atoms), len(idx)))
    n_irr = len(idx)
    if len(idx) == 0:
        np.random.seed(81396)
        idx = np.random.randint(low=0, high=len(atoms), size=10000)

    coords_all = atoms.get_positions()
    coords_sub = coords_all[idx]

    obj = utils.load_file("data/stat_indxs_cutoff_150/dump.arc.{}_stat_irrCoords.pkl".format(run))
    

    
    coords_stat = astat.get_realStats(coords_sub, coords_sub, obj["indexes"], r_stat=r_stat, 
                            cell=atoms.get_cell(), 
                            pbc=atoms.get_pbc())
    
    rdf, pdf, bin_centers = astat.get_rdf(coords_stat, r_stat, len_pixel*10)
    box = astat.get_2ptStat(coords_stat, r_stat, len_pixel)
    
    d = {}
    d["rdf"] = rdf
    d["pdf"] = pdf
    d["bins"] = bin_centers
    d["box"] = box
    d["len_pixel"] = len_pixel
    d["n_irr"] = n_irr
    
    utils.save_file(fname="{}/dump.arc.{}_stat.pkl".format(folder_rdf, run), obj=d)

In [19]:
%%time
import multiprocessing as mp
n_proc = 10
with mp.Pool(n_proc) as p:
    p.map(generate_stats(r_stat=r_stat, len_pixel=len_pixel), runs)

total atoms: 4000000, disturbed atoms: 244316
total atoms: 4000000, disturbed atoms: 132505
total atoms: 4000000, disturbed atoms: 272163
total atoms: 4000000, disturbed atoms: 207399
total atoms: 4000000, disturbed atoms: 289464
total atoms: 4000000, disturbed atoms: 0
total atoms: 4000000, disturbed atoms: 259220
total atoms: 4000000, disturbed atoms: 182610
total atoms: 4000000, disturbed atoms: 266384
total atoms: 4000000, disturbed atoms: 205043
total atoms: 4000000, disturbed atoms: 87589
total atoms: 4000000, disturbed atoms: 252704
total atoms: 4000000, disturbed atoms: 171079
total atoms: 4000000, disturbed atoms: 242254
total atoms: 4000000, disturbed atoms: 189780
total atoms: 4000000, disturbed atoms: 206487
total atoms: 4000000, disturbed atoms: 274957
total atoms: 4000000, disturbed atoms: 261720
total atoms: 4000000, disturbed atoms: 254992
total atoms: 4000000, disturbed atoms: 259587
total atoms: 4000000, disturbed atoms: 277355
total atoms: 4000000, disturbed atoms: 2